In [201]:
#create spark session
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('nlp').getOrCreate()

In [202]:
df=spark.createDataFrame([(1,'I really liked this movie'),
                         (2,'I would recommend this movie to my friends'),
                         (3,'movie was alright but acting was horrible'),
                         (4,'I am never watching that movie ever again')],
                        ['user_id','review'])

In [203]:
df.show(5,False)

+-------+------------------------------------------+
|user_id|review                                    |
+-------+------------------------------------------+
|1      |I really liked this movie                 |
|2      |I would recommend this movie to my friends|
|3      |movie was alright but acting was horrible |
|4      |I am never watching that movie ever again |
+-------+------------------------------------------+



In [ ]:
# Tokenization

In [204]:
from pyspark.ml.feature import Tokenizer

In [205]:
tokenization=Tokenizer(inputCol='review',outputCol='tokens')

In [206]:
tokenized_df=tokenization.transform(df)

In [207]:
tokenized_df.show(4,False)

+-------+------------------------------------------+---------------------------------------------------+
|user_id|review                                    |tokens                                             |
+-------+------------------------------------------+---------------------------------------------------+
|1      |I really liked this movie                 |[i, really, liked, this, movie]                    |
|2      |I would recommend this movie to my friends|[i, would, recommend, this, movie, to, my, friends]|
|3      |movie was alright but acting was horrible |[movie, was, alright, but, acting, was, horrible]  |
|4      |I am never watching that movie ever again |[i, am, never, watching, that, movie, ever, again] |
+-------+------------------------------------------+---------------------------------------------------+



In [ ]:
# stopwords removal 

In [208]:
from pyspark.ml.feature import StopWordsRemover

In [209]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [210]:
refined_df=stopword_removal.transform(tokenized_df)

In [211]:
refined_df.select(['user_id','tokens','refined_tokens']).show(10,False)

+-------+---------------------------------------------------+----------------------------------+
|user_id|tokens                                             |refined_tokens                    |
+-------+---------------------------------------------------+----------------------------------+
|1      |[i, really, liked, this, movie]                    |[really, liked, movie]            |
|2      |[i, would, recommend, this, movie, to, my, friends]|[recommend, movie, friends]       |
|3      |[movie, was, alright, but, acting, was, horrible]  |[movie, alright, acting, horrible]|
|4      |[i, am, never, watching, that, movie, ever, again] |[never, watching, movie, ever]    |
+-------+---------------------------------------------------+----------------------------------+



In [ ]:
# Count Vectorizer

In [212]:
from pyspark.ml.feature import CountVectorizer

In [213]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [214]:
cv_df=count_vec.fit(refined_df).transform(refined_df)

In [215]:
cv_df.select(['user_id','refined_tokens','features']).show(4,False)

+-------+----------------------------------+--------------------------------+
|user_id|refined_tokens                    |features                        |
+-------+----------------------------------+--------------------------------+
|1      |[really, liked, movie]            |(11,[0,4,9],[1.0,1.0,1.0])      |
|2      |[recommend, movie, friends]       |(11,[0,6,10],[1.0,1.0,1.0])     |
|3      |[movie, alright, acting, horrible]|(11,[0,2,3,5],[1.0,1.0,1.0,1.0])|
|4      |[never, watching, movie, ever]    |(11,[0,1,7,8],[1.0,1.0,1.0,1.0])|
+-------+----------------------------------+--------------------------------+



In [218]:
count_vec.fit(refined_df).vocabulary

['movie',
 'horrible',
 'really',
 'alright',
 'liked',
 'friends',
 'recommend',
 'never',
 'ever',
 'acting',
 'watching']

In [ ]:
#Tf-idf

In [220]:
from pyspark.ml.feature import HashingTF,IDF

In [221]:
hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features')

In [222]:
hashing_df=hashing_vec.transform(refined_df)

In [225]:
hashing_df.select(['user_id','refined_tokens','tf_features']).show(4,False)

+-------+----------------------------------+-------------------------------------------------------+
|user_id|refined_tokens                    |tf_features                                            |
+-------+----------------------------------+-------------------------------------------------------+
|1      |[really, liked, movie]            |(262144,[14,32675,155321],[1.0,1.0,1.0])               |
|2      |[recommend, movie, friends]       |(262144,[129613,155321,222394],[1.0,1.0,1.0])          |
|3      |[movie, alright, acting, horrible]|(262144,[80824,155321,236263,240286],[1.0,1.0,1.0,1.0])|
|4      |[never, watching, movie, ever]    |(262144,[63139,155321,203802,245806],[1.0,1.0,1.0,1.0])|
+-------+----------------------------------+-------------------------------------------------------+



In [227]:
tf_idf_vec=IDF(inputCol='tf_features',outputCol='tf_idf_features')

In [228]:
tf_idf_df=tf_idf_vec.fit(hashing_df).transform(hashing_df)

In [230]:
tf_idf_df.select(['user_id','tf_idf_features']).show(4,False)

+-------+----------------------------------------------------------------------------------------------------+
|user_id|tf_idf_features                                                                                     |
+-------+----------------------------------------------------------------------------------------------------+
|1      |(262144,[14,32675,155321],[0.9162907318741551,0.9162907318741551,0.0])                              |
|2      |(262144,[129613,155321,222394],[0.9162907318741551,0.0,0.9162907318741551])                         |
|3      |(262144,[80824,155321,236263,240286],[0.9162907318741551,0.0,0.9162907318741551,0.9162907318741551])|
|4      |(262144,[63139,155321,203802,245806],[0.9162907318741551,0.0,0.9162907318741551,0.9162907318741551])|
+-------+----------------------------------------------------------------------------------------------------+



In [ ]:
# Classification 

In [241]:
text_df=spark.read.csv('Movie_reviews.csv',inferSchema=True,header=True,sep=',')

In [242]:
text_df.printSchema()

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [243]:
text_df.count()

7087

In [244]:
from pyspark.sql.functions import rand 

In [245]:
text_df.orderBy(rand()).show(10,False)

+------------------------------------------------------------------------+---------+
|Review                                                                  |Sentiment|
+------------------------------------------------------------------------+---------+
|My dad's being stupid about brokeback mountain...                       |0        |
|Ok brokeback mountain is such a horrible movie.                         |0        |
|I love Brokeback Mountain.                                              |1        |
|He's like,'YEAH I GOT ACNE AND I LOVE BROKEBACK MOUNTAIN '..            |1        |
|Harry Potter and the Sorcerer's Stone is great but I had forgotten what |1        |
|"Anyway, thats why I love "" Brokeback Mountain."                       |1        |
|Which is why i said silent hill turned into reality coz i was hella like|1        |
|Apparently the Da Vinci code sucks.                                     |0        |
|I am going to start reading the Harry Potter series again becaus

In [246]:
text_df=text_df.filter(((text_df.Sentiment =='1') | (text_df.Sentiment =='0')))

In [247]:
text_df.count()

6990

In [248]:
text_df.groupBy('Sentiment').count().show()

+---------+-----+
|Sentiment|count|
+---------+-----+
|        0| 3081|
|        1| 3909|
+---------+-----+



In [249]:
text_df.printSchema()

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [250]:
text_df = text_df.withColumn("Label", text_df.Sentiment.cast('float')).drop('Sentiment')

In [251]:
text_df.orderBy(rand()).show(10,False)

+------------------------------------------------------------------------+-----+
|Review                                                                  |Label|
+------------------------------------------------------------------------+-----+
|I hate Harry Potter.                                                    |0.0  |
|I am the only person in the world who thought Brokeback Mountain sucked.|0.0  |
|Not because I hate Harry Potter, but because I am the type of person tha|0.0  |
|Which is why i said silent hill turned into reality coz i was hella like|1.0  |
|The Da Vinci Code sucked big time.                                      |0.0  |
|The Da Vinci Code sucked big time.                                      |0.0  |
|Ok brokeback mountain is such a horrible movie.                         |0.0  |
|A / N: This is a gift for sivullinen who requested: ” I'd love some NC  |1.0  |
|, she helped me bobbypin my insanely cool hat to my head, and she laughe|0.0  |
|I love the Da Vinci Code.  

In [252]:
text_df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  1.0| 3909|
|  0.0| 3081|
+-----+-----+



In [253]:
# Add length to the dataframe
from pyspark.sql.functions import length

In [254]:
text_df=text_df.withColumn('length',length(text_df['Review']))

In [255]:
text_df.orderBy(rand()).show(10,False)

+------------------------------------------------------------------------+-----+------+
|Review                                                                  |Label|length|
+------------------------------------------------------------------------+-----+------+
|I have to say that I loved Brokeback Mountain.                          |1.0  |46    |
|The Da Vinci Code is awesome!!                                          |1.0  |30    |
|Oh, and Brokeback Mountain was a terrible movie.                        |0.0  |48    |
|man i loved brokeback mountain!                                         |1.0  |31    |
|Even though Brokeback Mountain is one of the most depressing movies, eve|0.0  |72    |
|da vinci code sucks...                                                  |0.0  |22    |
|Combining the opinion / review from Gary and Gin Zen, The Da Vinci Code |0.0  |71    |
|da vinci code sucks...                                                  |0.0  |22    |
|Finally feel up to making the l

In [256]:
text_df.groupBy('Label').agg({'Length':'mean'}).show()

+-----+-----------------+
|Label|      avg(Length)|
+-----+-----------------+
|  1.0|47.61882834484523|
|  0.0|50.95845504706264|
+-----+-----------------+



In [108]:
# Data Cleaning

In [257]:
tokenization=Tokenizer(inputCol='Review',outputCol='tokens')

In [258]:
tokenized_df=tokenization.transform(text_df)

In [259]:
tokenized_df.show()

+--------------------+-----+------+--------------------+
|              Review|Label|length|              tokens|
+--------------------+-----+------+--------------------+
|The Da Vinci Code...|  1.0|    39|[the, da, vinci, ...|
|this was the firs...|  1.0|    72|[this, was, the, ...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|
|I liked the Da Vi...|  1.0|    72|[i, liked, the, d...|
|that's not even a...|  1.0|    72|[that's, not, eve...|
|I loved the Da Vi...|  1.0|    72|[i, loved, the, d...|
|i thought da vinc...|  1.0|    57|[i, thought, da, ...|
|The Da Vinci Code...|  1.0|    45|[the, da, vinci, ...|
|I thought the Da ...|  1.0|    51|[i, thought, the,...|
|The Da Vinci Code...|  1.0|    68|[the, da, vinci, ...|
|The Da Vinci Code...|  1.0|    62|[the, da, vinci, ...|
|then I turn on th...|  1.0|    66|[then, i, turn, o...|
|The Da Vinci Code...|  1.0|    34|[the, da, vinci, ...|
|i love da vinci c...|  1.0|   

In [260]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [261]:
refined_text_df=stopword_removal.transform(tokenized_df)

In [262]:
refined_text_df.show()

+--------------------+-----+------+--------------------+--------------------+
|              Review|Label|length|              tokens|      refined_tokens|
+--------------------+-----+------+--------------------+--------------------+
|The Da Vinci Code...|  1.0|    39|[the, da, vinci, ...|[da, vinci, code,...|
|this was the firs...|  1.0|    72|[this, was, the, ...|[first, clive, cu...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|[liked, da, vinci...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|[liked, da, vinci...|
|I liked the Da Vi...|  1.0|    72|[i, liked, the, d...|[liked, da, vinci...|
|that's not even a...|  1.0|    72|[that's, not, eve...|[even, exaggerati...|
|I loved the Da Vi...|  1.0|    72|[i, loved, the, d...|[loved, da, vinci...|
|i thought da vinc...|  1.0|    57|[i, thought, da, ...|[thought, da, vin...|
|The Da Vinci Code...|  1.0|    45|[the, da, vinci, ...|[da, vinci, code,...|
|I thought the Da ...|  1.0|    51|[i, thought, the,...|[thought

In [263]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [264]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_text_df = refined_text_df.withColumn("token_count", len_udf(col('refined_tokens')))


In [268]:
refined_text_df.orderBy(rand()).show(10)

+--------------------+-----+------+--------------------+--------------------+-----------+
|              Review|Label|length|              tokens|      refined_tokens|token_count|
+--------------------+-----+------+--------------------+--------------------+-----------+
|da vinci code was...|  1.0|    37|[da, vinci, code,...|[da, vinci, code,...|          5|
|Not because I hat...|  0.0|    72|[not, because, i,...|[hate, harry, pot...|          6|
|I love Harry Potter.|  1.0|    20|[i, love, harry, ...|[love, harry, pot...|          3|
|and I love Da Vin...|  1.0|    71|[and, i, love, da...|[love, da, vinci,...|          7|
|Da Vinci Code = U...|  0.0|    72|[da, vinci, code,...|[da, vinci, code,...|         15|
|Brokeback Mountai...|  1.0|    34|[brokeback, mount...|[brokeback, mount...|          3|
|I think I hate Ha...|  0.0|    72|[i, think, i, hat...|[think, hate, har...|          9|
|Harry Potter is b...|  1.0|    26|[harry, potter, i...|[harry, potter, b...|          3|
|The Da Vi

In [163]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [164]:
cv_text_df=count_vec.fit(refined_text_df).transform(refined_text_df)

In [270]:
cv_text_df.select(['refined_tokens','token_count','features','Label']).show(10)

+--------------------+-----------+--------------------+-----+
|      refined_tokens|token_count|            features|Label|
+--------------------+-----------+--------------------+-----+
|[da, vinci, code,...|          5|(2302,[0,1,4,43,2...|  1.0|
|[first, clive, cu...|          9|(2302,[11,51,229,...|  1.0|
|[liked, da, vinci...|          5|(2302,[0,1,4,53,3...|  1.0|
|[liked, da, vinci...|          5|(2302,[0,1,4,53,3...|  1.0|
|[liked, da, vinci...|          8|(2302,[0,1,4,53,6...|  1.0|
|[even, exaggerati...|          6|(2302,[46,229,271...|  1.0|
|[loved, da, vinci...|          8|(2302,[0,1,22,30,...|  1.0|
|[thought, da, vin...|          7|(2302,[0,1,4,228,...|  1.0|
|[da, vinci, code,...|          6|(2302,[0,1,4,33,2...|  1.0|
|[thought, da, vin...|          7|(2302,[0,1,4,223,...|  1.0|
+--------------------+-----------+--------------------+-----+
only showing top 10 rows



In [271]:
#select data for building model
model_text_df=cv_text_df.select(['features','token_count','Label'])

In [272]:
from pyspark.ml.feature import VectorAssembler

In [273]:
df_assembler = VectorAssembler(inputCols=['features','token_count'],outputCol='features_vec')
model_text_df = df_assembler.transform(model_text_df)

In [274]:
model_text_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- token_count: integer (nullable = true)
 |-- Label: float (nullable = true)
 |-- features_vec: vector (nullable = true)



In [276]:
from pyspark.ml.classification import LogisticRegression

In [277]:
#split the data 
training_df,test_df=model_text_df.randomSplit([0.75,0.25])

In [278]:
training_df.groupBy('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|  1.0| 2979|
|  0.0| 2335|
+-----+-----+



In [279]:
test_df.groupBy('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|  1.0|  930|
|  0.0|  746|
+-----+-----+



In [294]:
log_reg=LogisticRegression(featuresCol='features_vec',labelCol='Label').fit(training_df)

In [295]:
results=log_reg.evaluate(test_df).predictions

In [296]:
results.show()

+--------------------+-----------+-----+--------------------+--------------------+--------------------+----------+
|            features|token_count|Label|        features_vec|       rawPrediction|         probability|prediction|
+--------------------+-----------+-----+--------------------+--------------------+--------------------+----------+
|(2302,[0,1,4,5,64...|          6|  1.0|(2303,[0,1,4,5,64...|[-17.272830422692...|[3.15141100218827...|       1.0|
|(2302,[0,1,4,5,89...|          9|  1.0|(2303,[0,1,4,5,89...|[-5.3071943841355...|[0.00493137238287...|       1.0|
|(2302,[0,1,4,5,30...|          5|  1.0|(2303,[0,1,4,5,30...|[-20.050569575912...|[1.95951356060452...|       1.0|
|(2302,[0,1,4,5,44...|          5|  1.0|(2303,[0,1,4,5,44...|[-20.154922616911...|[1.76533984442990...|       1.0|
|(2302,[0,1,4,5,82...|          6|  1.0|(2303,[0,1,4,5,82...|[-14.417812465440...|[5.47549475575723...|       1.0|
|(2302,[0,1,4,11,1...|          6|  0.0|(2303,[0,1,4,11,1...|[19.1666519710833..

In [297]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [298]:
#confusion matrix
true_postives = results[(results.Label == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Label == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Label == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Label == 1) & (results.prediction == 0)].count()

In [299]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.986021505376344


In [300]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.9572025052192067


In [301]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)

0.9677804295942721
